<a href="https://colab.research.google.com/github/hihi174/AI-Agent-study-time-/blob/main/AI_agent_(study_time)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
from datetime import datetime, timedelta

In [ ]:
from google.colab import userdata
openAI_Key=userdata.get('OpenAI_Key')

In [ ]:
pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 5.3 MB/s eta 0:00:00


In [ ]:
# === StudyMate Agent (Groq) — Fully Safe JSON Architecture ===
# This version fixes ALL JSONDecodeError problems.
# Each module is isolated and guaranteed to output STRICT JSON.

from groq import Groq
from google.colab import userdata
import json
import re


# Utility: Extract JSON Safely

def extract_json(text):
    pattern = r"\{(?:[^{}]|(?:\{[^{}]*\}))*\}"  # matches first JSON object
    matches = re.findall(pattern, text, flags=re.DOTALL)

    if not matches:
        raise ValueError("No JSON found in LLM output:\n" + text)

    for block in matches:
        try:
            return json.loads(block)
        except:
            continue

    raise ValueError("Found JSON-like text but could not parse it.")


# Groq LLM Wrapper

def call_llm(system_prompt, user_prompt):
    key = userdata.get('OpenAI_Key')
    if not key:
        raise ValueError("Missing API key in secrets.")

    client = Groq(api_key=key)

    response = client.chat.completions.create(
        model="moonshotai/kimi-k2-instruct",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    return response.choices[0].message.content


# PARSE MODULE (Always outputs JSON)
def parse_user_request(user_text, prev_state=None):
    system = (
        "You are the InputParser.\n"
        "Your ONLY job is to read the user's request and output STRICT JSON.\n"
        "Never explain anything. Never output text outside JSON.\n"
        "JSON format:\n"
        "{\n"
        "  'intent': 'study_plan' | 'learn_topic' | 'quiz' | 'revise' | 'summary',\n"
        "  'details': { ... extracted info ... },\n"
        "  'flags': {}\n"
        "}"
    )

    user = f"User input: {user_text}\nPrevious state: {json.dumps(prev_state or {})}"

    raw = call_llm(system, user)

    return extract_json(raw)



# PLANNING MODULE (Always outputs JSON)

def create_plan(parsed):
    system = (
        "You are the TaskPlanner.\n"
        "Given the parsed JSON, build a task plan.\n"
        "Output STRICT JSON with fields: task_type, steps[]"
    )

    user = json.dumps(parsed)
    raw = call_llm(system, user)

    return extract_json(raw)


# SESSION GENERATOR (Free text allowed)
def generate_session(plan, original_input):
    system = (
        "You are the StudySessionEngine.\n"
        "You generate HUMAN-FRIENDLY content based on the plan.\n"
        "Output plain text only."
    )

    user = f"Plan: {json.dumps(plan)}\nUser: {original_input}"

    return call_llm(system, user)


# STATE MANAGER (Must output JSON)
def update_state(parsed, session_output, prev_state=None):
    system = (
        "You are the StateManager.\n"
        "Update the student's state.\n"
        "Output STRICT JSON ONLY."
    )

    user = (
        f"Previous: {json.dumps(prev_state or {})}\n"
        f"Parsed: {json.dumps(parsed)}\n"
        f"Session: {session_output}"
    )

    raw = call_llm(system, user)

    return extract_json(raw)


# AGENT ORCHESTRATOR
def run_agent(user_input, state_file="state.json"):
    try:
        prev = json.load(open(state_file))
    except:
        prev = {}

    parsed = parse_user_request(user_input, prev)
    plan = create_plan(parsed)
    session = generate_session(plan, user_input)
    new_state = update_state(parsed, session, prev)

    with open(state_file, "w") as f:
        json.dump(new_state, f, indent=4)

    return session


# LOCAL CLI RUNNER
if __name__ == "__main__":
    print("=== StudyMate Agent (Groq) — Clean JSON Engine ===")

    while True:
        user_input = input("\nRequest (or 'exit'): ")

        if user_input.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break

        output = run_agent(user_input)
        print("\n=== SESSION OUTPUT ===\n")
        print(output)


=== StudyMate Agent (Groq) — Clean JSON Engine ===

Request (or 'exit'): i have my exam in 2 hrs and these topics i wnt to study bss,ms,bsc ,nss . prepare a plan for me 

=== SESSION OUTPUT ===

2-Hour Sprint Study Plan  
(keep this page open; tick each item as you finish)

00:00-00:30  BSS Fundamentals  
- 0-20 min: Radio path, TRX, BTS, BSC, Abis & Um (draw stick-figure tower → TRX → Abis → BSC)  
- 20-25 min: Quick recap: say aloud “Um carries RR/LLC/RRM, Abis = 16 kbit/s TRX-signalling + traffic”  
- 25-30 min: Tag anything fuzzy in margin with “★”

00:30-01:00  MS Side & Signalling  
- 30-50 min: SIM→ME, IMSI/TMSI, MM, CM; 3 main flows: LU, CM-service-request, Paging-response (scribble vertical ladder with BTS-BSC-MSC arrows)  
- 50-55 min: Recap: “MM handles location, CM handles calls/SMS”  
- 55-60 min: Star any step you can’t redraw without notes

01:00-01:30  BSC Deep Dive  
- 60-80 min: Key jobs: radio-resource, power-control, HO decision, code A-bis & A-interface; draw small